# Natural Language Queries for SQL using SLIM (Full End-to-End Example)

### Introduction

You've might have heard of SQL. It's a widely used programming language for storing and processing information in relational databases - simply put, relational databases store data in tables, where each row stores an entity and each column stores an attribute for that entity.

Let's say we have a table called customers in a relational database. If I wanted to access the names of all customers (customer_names) that have an annual_spend of at least $1000, I would have to formulate an SQL query like this:
```SQL
SELECT customer_names FROM customers WHERE annual_spend >= 1000
```
I would then run this query against the database to access my results.

But what if AI 🤖 could do all this for us?

LLMWare allows us to do just that, making use of small language models. These are models of a smaller scale, with fewer and less precise parameters that don't require much computational power to run. Their main advantage is that they run locally on a CPU, without an internet connection or a GPU. This enables use to make our queries entirely in natural language and still get accurate results!

Let's look at an example of how to do this from start to finish.

### For Google Colab users

If you are using Colab for free, we highly recommend you activate the T4 GPU hardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as apposed to the ~13GB Colab gives automatically.

To activate T4:
1. click on the "Runtime" tab
2. click on "Change runtime type"
3. select T4 GPU under Hardware Accelerator

NOTE: there is a weekly usage limit on using T4 for free

### Installing and importing dependencies

In [1]:
%pip install llmware

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from llmware.agents import SQLTables, LLMfx
from llmware.models import ModelCatalog
from llmware.configs import LLMWareConfig

### Worker function

The process to generate an output for the given input query is as follows:
1. If `create_new_table` is set to `True`, then:
    - Check to see if the SLIM SQL tool is already downloaded, if not, download it using the `ModelCatalog` class
    - Load in the sample CSV file `customer_table.csv`
    - Create an SQL table from the CSV file using the `SQLTables` class
2. Create an agent, an instance of the `LLMfx` class, and load in the SQL tool
3. For each query in the `query_list`, call the `query_db` function on the agent, which:
    - looks up the table schema in the db using the table_name
    - packages the text-2-sql query prompt
    - executes sql method to convert the prompt into a sql query
    - attempts to execute the sql query on the db
    - returns the db results as 'research' output
4. Print the result from the `agent`'s `research_list`.

In [3]:
def sql_e2e_test_script(table_name="customers1",create_new_table=False):

    """ This is the end-to-end execution script. """

    #   create table if needed to set up
    if create_new_table:

        # looks to pull sample csv 'customer_table.csv' from slim-sql-tool model package files
        sql_tool_repo_path = os.path.join(LLMWareConfig().get_model_repo_path(), "slim-sql-tool")

        if not os.path.exists(sql_tool_repo_path):
            ModelCatalog().load_model("llmware/slim-sql-tool")

        files = os.listdir(sql_tool_repo_path)
        csv_file = "customer_table.csv"

        if csv_file in files:

            #   to create a testing table from a csv
            sql_db = SQLTables(experimental=True)
            sql_db.create_new_table_from_csv(sql_tool_repo_path, csv_file, table_name=table_name)
            #   end - creating table

            print("update: successfully created new db table")
        else:
            print("something has gone wrong - could not find customer_table.csv inside the slim-sql-tool file package")

    #   query starts here
    agent = LLMfx()
    agent.load_tool("sql", sample=False, get_logits=True, temperature=0.0)

    #  Pass direct queries to the DB

    query_list = ["Which customers have vip customer status of yes?",
                  "What is the highest annual spend of any customer?",
                  "Which customer has account number 1234953",
                  "Which customer has the lowest annual spend?",
                  "Is Susan Soinsin a vip customer?"]

    for i, query in enumerate(query_list):

        #   query_db method is doing all of the work
        #   -- looks up the table schema in the db using the table_name
        #   -- packages the text-2-sql query prompt
        #   -- executes sql method to convert the prompt into a sql query
        #   -- attempts to execute the sql query on the db
        #   -- returns the db results as 'research' output

        response = agent.query_db(query, table=table_name)

    for x in range(0,len(agent.research_list)):
        print("research: ", x, agent.research_list[x])

    return 0

### Helper functions

If you want to delete a table in the experimental database, you can use the `delete_table()` function.

In [4]:
def delete_table(table_name):

    """ Start fresh in testing - delete table in experimental local SQLite DB """

    sql_db = SQLTables(experimental=True)
    sql_db.delete_table(table_name, confirm_delete=True)

    return True

If you want to delete the entire database, you use the `delete_db()` function.

In [5]:
def delete_db():

    """ Start fresh in testing - deletes SQLite DB and starts over. """

    sql_db = SQLTables(experimental=True)
    sql_db.delete_experimental_db(confirm_delete=True)

    return True

### Main block

We can now run our code by calling the `sql_e2e_test_script` function.

In [6]:
if __name__ == "__main__":

    ModelCatalog().get_llm_toolkit(tool_list=["sql"])

    #   run an end-to-end test
    sql_e2e_test_script(table_name="customer1",create_new_table=True)

    #   third - delete and start fresh for further testing
    delete_table("customer1")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

library_table.csv:   0%|          | 0.00/317 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

customer_table.csv:   0%|          | 0.00/929 [00:00<?, ?B/s]

slim-sql.gguf:   0%|          | 0.00/669M [00:00<?, ?B/s]

update: successfully created new db table
update: Launching LLMfx process
step - 	1 - 	creating object - ready to start processing.
step - 	2 - 	loading tool - sql
step - 	3 - 	loading new processing text - 1 new entries
step - 	4 - 	executing function call - deploying - text-to-sql
				 -- query - Which customers have vip customer status of yes?
				 -- table_schema - CREATE TABLE customer1 (customer_name text, account_number integer, customer_level text, vip_customer text, annual_spend integer, user_name text )
step - 	5 - 	executing function call - getting response - sql
				 -- llm_response - SELECT customer_name FROM customer1 WHERE vip_customer = 'yes'
				 -- output type - text
				 -- usage - {'input': 57, 'output': 16, 'total': 73, 'metric': 'tokens', 'processing_time': 0.877871036529541}
step - 	6 - 	executing research call - executing query on db
				 -- db - C:\Users\prash\llmware_data\accounts\sqlite_experimental.db
				 -- sql_query - SELECT customer_name FROM customer1 W

step - 	25 - 	executing function call - getting response - sql
				 -- llm_response - SELECT vip_customer FROM customer1 WHERE customer_name = 'Susan Soinsin'
				 -- output type - text
				 -- usage - {'input': 56, 'output': 21, 'total': 77, 'metric': 'tokens', 'processing_time': 0.6235740184783936}
step - 	26 - 	executing research call - executing query on db
				 -- db - C:\Users\prash\llmware_data\accounts\sqlite_experimental.db
				 -- sql_query - SELECT vip_customer FROM customer1 WHERE customer_name = 'Susan Soinsin'
step - 	27 - 	executing research  - getting response - sql
				 -- result - [('no',)]
research:  0 {'step': 6, 'tool': 'sql', 'db_response': [('Martha Williams',), ('John Jones',), ('Arvind Arora',), ('Vinod Aggarwal',), ('Allison Winters',), ('Olivia Smith',), ('Alan Wang',), ('Wilmer Rodriguez',), ('Eliza Listron',), ('Douglas Hudson',)], 'sql_query': "SELECT customer_name FROM customer1 WHERE vip_customer = 'yes'", 'query': 'Which customers have vip customer stat

We now have the output of our program. The output for each natural language question is a dictionary containing a lot of detailed information about the steps carried out by the agent, but here are some of the more interesting parts of it:
- `db_response` gives us what we want, the answer to the question we ask
- `sql_query` shows us the SQL query that was generated from our natural language question using the SLIM SQL tool.